# Spark SQL

Spark SQL is a module in Apache Spark for structured data processing. It allows you to run SQL queries on large datasets, combining the benefits of SQL with the scalability of Spark’s distributed computing capabilities.
- In this work, Yelp academic datasets are used for data manipulation using Spark SQL

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate() 

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/10/30 18:55:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


- Let's read our data which is json file of Yelp academic dataset

In [2]:
business = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('../../assets/data/yelp_academic/yelp_academic_dataset_user.json.gz')

24/10/30 18:55:26 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
type(business) # business is a dataframe.

pyspark.sql.dataframe.DataFrame

- To get ready to work with SQL commands on our Spark Session Data Frames, we can create a view onto our dataframes. Every Spark Session can have a view created. Either, temporary, permanent or global. 
- Here we consider a temporary view. Temporary views disappear after Spark Session ends. 
- Once we have a temp view, we can use it to issue  SQL querry view that we have created. 

# Create temp views for the DataFrames

In [4]:
business.createOrReplaceTempView("business")
checkin.createOrReplaceTempView("checkin")
tip.createOrReplaceTempView("tip")
review.createOrReplaceTempView("review")
user.createOrReplaceTempView("user")

In [5]:
# Let's also read our state csv file which refers to US States and create a temp view for that as well: 
us_states = spark.read.option("header",True).csv('../../assets/data/states.csv')
us_states.createOrReplaceTempView("us_states")

# -- elite review --

- Get a list of users named "Jenna" with the number of their reviews tagged "elite".

In [6]:
user.printSchema()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [7]:
user.show(2)

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+--------------+----+--------------------+-----+------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer|cool|         elite|fans|             friends|funny|  name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+----+--------------+----+--------------------+-----+------+------------+------+--------------------+-------------------+
|         4.03|              1|         

In [8]:
query = """
SELECT user_id, name, fans 
FROM user
WHERE name ='Jenna'
ORDER BY fans DESC
"""

result = spark.sql(query)

In [9]:
result.show(5)

+--------------------+-----+----+
|             user_id| name|fans|
+--------------------+-----+----+
|bEX-LmV3ViAbv7mK_...|Jenna| 115|
|VRvD7-JHdWdTQnh1l...|Jenna| 100|
|QpbySlIOUOkrT9YeV...|Jenna|  93|
|kxj7TPtJY2zbEjGV0...|Jenna|  84|
|WH612ezymlanNW5oB...|Jenna|  67|
+--------------------+-----+----+
only showing top 5 rows



# -- USERS WITH    more than 1000     FANS --

- Let's find out the number of users that have more than 1000 fans.

In [10]:
# Using SPARK DataFrame functions:
user.filter(user['fans']>1000).count()

46

In [11]:
query = """
SELECT count(user_id)
FROM user
WHERE fans > 1000
"""

result = spark.sql(query)

In [12]:
result.show()

+--------------+
|count(user_id)|
+--------------+
|            46|
+--------------+



# -- Review Star Business --

- Let's find out the number of businesses with at least 4 stars and at least 100 review_count.



In [13]:
business.show(2)

+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+-------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|       city|               hours|is_open|     latitude|    longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+-------+-------------+-------------+--------------------+-----------+------------+-----+-----+
|2818 E Camino Ace...|{null, null, null...|1SWheh84yJXfytovI...|   Golf, Active Life|    Phoenix|                null|      0|   33.5221425| -112.0184807|Arizona Biltmore ...|      85016|           5|  3.0|   AZ|
|30 Eglinton Avenue W|{null, null, u'fu...|QXAEGFB4oINsVuTFx...|Specialty Food, R...|Mississauga|{9:0-1:0, 9:0-0:0...|      1|43.6054989743|-79.6522

In [14]:
query = """
SELECT count(business_id) AS Num_business
FROM business
WHERE review_count >= 100 AND stars >= 4
"""
result = spark.sql(query)

In [15]:
result.show()

+------------+
|Num_business|
+------------+
|        7464|
+------------+



# -- LITCHFIELD OHIO --

- Get a list of businesses from Litchfield, OH.

In [16]:
business.columns

['address',
 'attributes',
 'business_id',
 'categories',
 'city',
 'hours',
 'is_open',
 'latitude',
 'longitude',
 'name',
 'postal_code',
 'review_count',
 'stars',
 'state']

In [17]:
def OH_business():
    return spark.sql("""\
    SELECT business_id, name, city, state
    FROM business
    WHERE city = 'Litchfield' AND state = 'OH'
    """
    )

In [18]:
OH_Business_sql = OH_business()
OH_Business_sql.show()

+--------------------+--------------------+----------+-----+
|         business_id|                name|      city|state|
+--------------------+--------------------+----------+-----+
|ucY9cyJinEaPfXZWA...|   T.L. Keller Meats|Litchfield|   OH|
|t1w3AgvdlXCGAaTTT...|       Gizmos Grille|Litchfield|   OH|
|_sRy_DqdGrUWqqYsr...|Old School House ...|Litchfield|   OH|
|bRS8vlTZmdGGxF9vi...|           Gary Mart|Litchfield|   OH|
|3NzMmFv3Ky_cxtL9Y...|        Tonios Pizza|Litchfield|   OH|
|CFs89vckryf0WNb6z...|         Hungry Bear|Litchfield|   OH|
+--------------------+--------------------+----------+-----+



# -- US STATES IN BUSINESS --

- Find out which US states are represented in the business data set. 

In [19]:
def state_function ():
    return spark.sql("""
        SELECT DISTINCT us_states.state
        FROM us_states
        JOIN business 
        ON business.state = us_states.abbreviation
        """)


In [20]:
state_business = state_function()
state_business.show()

+--------------+
|         state|
+--------------+
|          Utah|
|          Ohio|
|      Arkansas|
|         Texas|
|  Pennsylvania|
|   Connecticut|
|      Nebraska|
|       Vermont|
|        Nevada|
|    Washington|
|      Illinois|
|        Alaska|
|    New Mexico|
|       Georgia|
|      Virginia|
|North Carolina|
|    New Jersey|
|       Alabama|
|       Arizona|
|     Tennessee|
+--------------+
only showing top 20 rows



# -- FUNNIEST REVIEW --

- Let's retrieve the text of the funniest review.

In [21]:
review.select('funny').show(3)

+-----+
|funny|
+-----+
|    1|
|    0|
|    0|
+-----+
only showing top 3 rows



In [22]:
def funniest_review():
    return spark.sql("""\
            SELECT text  
            FROM review
            WHERE funny = (SELECT max(funny) from review)
            """)

In [23]:
funniest_rev = funniest_review()
funniest_rev.show()

+--------------------+
|                text|
+--------------------+
|Flew to Arizona a...|
+--------------------+



## -- NAME OF MOST TIPS --

- Let's find the names of the top 100 users who provided the most tips.
- The result should be sorted by highest-to-lowest tip_count. However, in the case of tip_count ties, the results should be sorted by name alphabetically. 

In [24]:
user.columns

['average_stars',
 'compliment_cool',
 'compliment_cute',
 'compliment_funny',
 'compliment_hot',
 'compliment_list',
 'compliment_more',
 'compliment_note',
 'compliment_photos',
 'compliment_plain',
 'compliment_profile',
 'compliment_writer',
 'cool',
 'elite',
 'fans',
 'friends',
 'funny',
 'name',
 'review_count',
 'useful',
 'user_id',
 'yelping_since']

In [25]:
tip.columns

['business_id', 'compliment_count', 'date', 'text', 'user_id']

In [26]:
query = """
    SELECT user.name, user.user_id, count(tip.date) AS tip_count 
    FROM user
    JOIN tip ON tip.user_id=user.user_id
    GROUP BY user.name, user.user_id
    ORDER BY tip_count DESC, name ASC"""

result = spark.sql(query)

In [27]:
result.show()

+--------+--------------------+---------+
|    name|             user_id|tip_count|
+--------+--------------------+---------+
|    Momo|mkbx55W8B8aPLgDqe...|     2439|
|Jennifer|CxDOIDnH8gp9KXzpB...|     1598|
|Samantha|6ZC-0LfOAGwaFc5XP...|     1509|
|  Daniel|0tvCcnfJnSs55iB6m...|     1376|
|Christie|eZfHm0qI8A_HfvXSc...|     1352|
|     May|O8eDScRAg6ae0l9Bc...|     1255|
|   Kurdy|8DGFWco9VeBAxjqsu...|     1178|
| Anthony|WJKocp9RE0KatUwh3...|     1161|
| Shirley|2EuPAGalYnP7eSxPg...|     1154|
| Cherrie|QPJJohtGqkMkaN0Gt...|     1017|
|  Nelson|3nDUQBjKyVor5wV0r...|     1002|
|    John|5dKknvq65x-SaluuJ...|      997|
| Georgie|A0j21z2Q1HGic7jW6...|      986|
|   Roger|0FMte0z-repSVWSJ_...|      925|
|   Vegas|uG35h72BAMutvXAWd...|      846|
|    Tony|RQlnSCjuqMnhR3Qk6...|      820|
| Stephen|tQPk4JiBPsx7NSIDb...|      794|
|  Alfred|Fw4UjJ6yBeyPB27Y4...|      793|
|  Arlene|fmzIm7RxEdii5Jz44...|      773|
| Colanie|pn_flI3EBNugBEYFp...|      771|
+--------+--------------------+---

In [28]:
def name_most_tips():
    return spark.sql("""
        SELECT user.name, count(user.user_id) as tip_count  
        FROM tip
        join user on user.user_id = tip.user_id
        group by name, user.user_id
        ORDER BY tip_count DESC, name ASC
        """)

most_tips = name_most_tips()
most_tips.show(5)

+--------+---------+
|    name|tip_count|
+--------+---------+
|    Momo|     2439|
|Jennifer|     1598|
|Samantha|     1509|
|  Daniel|     1376|
|Christie|     1352|
+--------+---------+
only showing top 5 rows



## -- NEW YORK SUMMARY -- 

- Let's figure out the list of user names and number of reviews of businesses in NewYork ('NY') 

In [29]:
def name_review():
    return spark.sql("""
        SELECT user.name, count(review.business_id) FILTER(WHERE state = 'NY') AS ny_count
        FROM user

        JOIN review ON review.user_id = user.user_id 

        JOIN business ON business.business_id = review.business_id

        GROUP BY user.name, user.user_id

        ORDER BY ny_count DESC"""
                    )



In [30]:
name_review = name_review()
name_review.show()

+-------+--------+
|   name|ny_count|
+-------+--------+
|  Stacy|       4|
|Stephen|       4|
|  Sarah|       4|
|    lou|       2|
|Matthew|       2|
| Rachel|       2|
|  Amber|       2|
|  Trina|       2|
|  Henry|       2|
| Nicole|       2|
|Katelyn|       2|
|   Skip|       2|
|  Megan|       2|
| Millie|       2|
|      R|       2|
|      K|       2|
|Mallory|       2|
|   Matt|       2|
|      G|       2|
|  Brian|       2|
+-------+--------+
only showing top 20 rows



## -- NEW YORK SUMMARY -- 

- Now, we can find the user names, number of reviews of businesses in New York ('NY') and total number of reviews of the top 10 users (as determined by who has created the most number of reviews of businesses in New York). Include a column that shows the percentage of reviews that are of businesses from New York. 

In [31]:
def name_review_percent():
    return spark.sql("""
            SELECT user.name, count(review.business_id) FILTER(WHERE state = 'NY') AS ny_count,
            user.review_count AS total_review,
            ROUND(count(review.business_id) FILTER(WHERE state = 'NY') *100 / (user.review_count), 2) AS percent

            FROM user

            JOIN review 
            ON review.user_id = user.user_id 

            JOIN business 
            ON review.business_id = business.business_id   

            GROUP BY user.name, user.user_id, user.review_count

            ORDER BY ny_count DESC, total_review DESC
            LIMIT 10"""
                    )
        


In [32]:
    
name_review = name_review_percent()
name_review.show(5)

+-------+--------+------------+-------+
|   name|ny_count|total_review|percent|
+-------+--------+------------+-------+
|  Sarah|       4|         337|   1.19|
|  Stacy|       4|           8|   50.0|
|Stephen|       4|           8|   50.0|
|  Amber|       2|         617|   0.32|
|Katelyn|       2|          78|   2.56|
+-------+--------+------------+-------+
only showing top 5 rows

